## Some ideas

1. Since we already have data that labelled as same question paris. Is it possible for us to train another model which can sequetially generate a same question string depends on the input? 
2. curriculum learning
3. Ensombling:

        a. Naive RNN
        b. xgboost
        c. randomforest
        
4. Feature engineering

        a. question length (May not be very powerful)
        b. similarity metric score (Directly count it and feed it as training feature. For exampole: cosine similarity LOL)
        c. I don't think TF-IDF would work, but still worth trying if we have time.
        
5. Create tags for each question by using the sentense paired with it. 

        For example:
        Q1: I am a good man.    <->    Q2: A good man is me.
        
        Q1 is tagged with [A, good, man, is, me]
        Q2 is tagged with [I, am, a, good, man]
        
        Then use these data to train a model predicting what tags would appear for each question. 

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import json
import matplotlib.pyplot as plt
import random

from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))

duplicated_qid_pair = pickle.load(open('../dataset/processed/positive_question_id_pairs.pkl', 'rb'))
qid_question_dict = pickle.load(open('../dataset/processed/qid_question_wrods_list_dict.pkl', 'rb'))

In [3]:
def enc_question(question, enc_map):
    
    if type(question)!=str or question=="":
        return [enc_map['<ST>'], enc_map['<ED>']]
    
    def map_wrod(word):
        return enc_map[word] if word in enc_map else enc_map['<RARE>']
         
    
    # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
    separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>]|$)'
    single_word = '[\S]+' # non-empty is enough here
    
    words_list = re.findall(single_word+separator, question)
    
    return [enc_map['<ST>']] + [map_wrod(word) for word in words_list] + [enc_map['<ED>']] 
    
    
def dec_question(question, dec_map):
    return [dec_map[enc_value] for enc_value in question]

## Clip training questions

Normalize the length of all question to 30

In [4]:
for key in qid_question_dict:
    q_len = len(qid_question_dict[key])
    if q_len>=30:
        qid_question_dict[key] = qid_question_dict[key][:30]
    else:
        qid_question_dict[key] = qid_question_dict[key] + ['<ED>'] * (30-q_len)

In [5]:
pickle.dump(qid_question_dict, open('../dataset/processed/qid_question_wrods_list_dict_clipped.pkl','wb'))

## Parse and encode testing data (this is very slow)

I still parses question length feature, although I haven't use it.

In [17]:
import warnings
warnings.filterwarnings('ignore')

counter = 0

def change_to_list_and_clip(data, clip_st_ed=False):
    
    global counter
    counter+=1
    if counter%10000==0:
        print('Changed',counter,'data')
    
    clip_len = 30
    data['q1_len'] = len(data['question1'])
    data['q2_len'] = len(data['question2'])

    if clip_st_ed:
        data['question1'] = data['question1'][1:-1]
        data['question2'] = data['question2'][1:-1]

    if len(data['question1'])<clip_len:
        data['question1'] = data['question1'] + [enc_map['<ED>']] * (30-data['q1_len'])
    else:
        data['question1'] = data['question1'][:30]
        
    if len(data['question2'])<clip_len:
        data['question2'] = data['question2'] + [enc_map['<ED>']] * (30-data['q2_len'])
    else:
        data['question2'] = data['question2'][:30]
    
    return data

In [2]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))
df_test = pickle.load(open('../dataset/processed/processed_testing_data.pkl','rb'))

In [19]:
%%time

counter = 0
df_transformed = df_test.apply(change_to_list_and_clip, axis=1)
pickle.dump(df_transformed, open('../dataset/processed/df_test_hubertLin_version.pkl', 'wb'))

Changed 10000 data
Changed 20000 data
Changed 30000 data
Changed 40000 data
Changed 50000 data
Changed 60000 data
Changed 70000 data
Changed 80000 data
Changed 90000 data
Changed 100000 data
Changed 110000 data
Changed 120000 data
Changed 130000 data
Changed 140000 data
Changed 150000 data
Changed 160000 data
Changed 170000 data
Changed 180000 data
Changed 190000 data
Changed 200000 data
Changed 210000 data
Changed 220000 data
Changed 230000 data
Changed 240000 data
Changed 250000 data
Changed 260000 data
Changed 270000 data
Changed 280000 data
Changed 290000 data
Changed 300000 data
Changed 310000 data
Changed 320000 data
Changed 330000 data
Changed 340000 data
Changed 350000 data
Changed 360000 data
Changed 370000 data
Changed 380000 data
Changed 390000 data
Changed 400000 data
Changed 410000 data
Changed 420000 data
Changed 430000 data
Changed 440000 data
Changed 450000 data
Changed 460000 data
Changed 470000 data
Changed 480000 data
Changed 490000 data
Changed 500000 data
Changed 5

# model time!

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import json
import matplotlib.pyplot as plt
import random

In [2]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))
embedding_matrix = pickle.load(open('../dataset/processed/embedding_matrix.pkl','rb'))

positive_question_pairs = pickle.load(open('../dataset/processed/positive_question_id_pairs.pkl', 'rb'))
negative_question_pairs = pickle.load(open('../dataset/processed/negative_question_id_pairs.pkl', 'rb'))

qid_question_dict = pickle.load(open('../dataset/processed/qid_question_wrods_list_dict_clipped.pkl','rb'))

In [3]:
# set to None if you don't want to do N_gram parsing/training (but you still need to modify the model's structure by yourself)
N_gram = 5

#### Train on all data directly. Overcome the imbalance problem by directly assign weights for each class label

In [4]:
test_set_pos_label_ratio = 0.1746

In [5]:
# remove validation data from training data
def holdout(qid_pair_list, ratio=0.025):
    
    def gen_suffle_idx(arr):
        random.shuffle(arr)
        return arr
    
    holdout_size = int(len(qid_pair_list)*ratio)
    idxes = gen_suffle_idx(np.arange(len(qid_pair_list)))
    holdout_idxes = idxes[:holdout_size]
    train_idxes = idxes[holdout_size:]
    
    train = qid_pair_list[train_idxes]
    val   = qid_pair_list[holdout_idxes]
    
    return train, val


In [6]:
# The validation data is heldout in the data augmenting phase, no need to holdout here

# train_pairs, val_pairs = holdout(train_pairs, ratio=0.01)

#### Process the validation set

In [7]:
# ex: [a,b,c] with N=3 =>  [ED,ED,A], [ED,A,B], [A,B,C], [B,C,ED], [C,ED,ED], where A,B,C,ED are embedded vectors
def N_gram_embedding(sample_lists, N):
    
    sample_size, question_len = sample_lists.shape
    sub_seq_len = question_len+N-1
    res = np.zeros((sample_size, sub_seq_len, N, 300))

    for s in range(sample_size):

        # insert ED
        res[s,0,0], res[s,0,1], res[s,1,0], res[s,-1,-1], res[s,-1,-2], res[s,-2,-1] = [embedding_matrix[enc_map['<ED>']]]*6

        for i,val in enumerate(sample_lists[s]):
            for n in range(N):
                if i+n<sub_seq_len and N-n-1>-1:
                    res[s,i+n,N-n-1] = embedding_matrix[val]

    return res

In [8]:
def process_validation_samples(df, enc_map, N_gram=None):
    
    def parse_word_list(question):
        
        if type(question)!=str:
            return []

        # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
        separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|$)'
        single_word = '[^\s\-]+' # non-empty is enough here

        return re.findall(single_word+separator, question)
    
    # encode single question pair
    def enc_question_pair(pair):

        # This array is a local cache that identify those non-encoded rare words. 
        # We'll assign identical index (in this question pair) for each of rare words
        rare_words = []

        def enc_question(question):

            def map_word(word):
                if word in enc_map:
                    return enc_map[word]
                elif word in rare_words:
                    idx = np.where(np.array(rare_words)==word)[0][0]
                    return enc_map['<RARE' + str(idx) + '>']
                else:
                    rare_words.append(word)
                    return enc_map['<RARE' + str(len(rare_words)-1) + '>']

            word_list = parse_word_list(question)
            word_list = [map_word(word) for word in word_list]
            
            # clip the question length to 30
            q_len = len(word_list)
            if q_len>=30:
                word_list = word_list[:30]
            else:
                word_list = word_list + [enc_map['<ED>']] * (30-q_len)
            
            return np.array(word_list)
        
        return [enc_question(pair[0]) , enc_question(pair[1])]
    
    X = np.array([enc_question_pair([series['question1'],series['question2']]) for i,series in df.iterrows()])
    y = np.array([series['is_duplicate'] for i,series in df.iterrows()])
    
    if N_gram==None:
        return [embedding_matrix[X[:,0]], embedding_matrix[X[:,1]]], y
    else:
        return [N_gram_embedding(X[:,0],N_gram),N_gram_embedding(X[:,1],N_gram)], y

In [9]:
df_val = pickle.load(open('../dataset/processed/validation_df.pkl','rb'))
val_data = process_validation_samples(df_val, enc_map, N_gram=N_gram)
del df_val

#### Calculate validation data's class weights

In [10]:
# Calculate the class weights to simulate the class disribution of testing samples
#
# These numbers and formula is got from here: 
#     https://www.kaggle.com/lystdo/quora-question-pairs/lstm-with-word2vec-embeddings#175198

test_set_pos_label_ratio = 0.1746

X_val = val_data[0]
y_val = val_data[1]
validation_pos_ratio = sum(y_val)/len(y_val)

val_weights = {
    0: (1-test_set_pos_label_ratio) / (1-validation_pos_ratio),
    1: test_set_pos_label_ratio/validation_pos_ratio
}

validation_weights = np.repeat(val_weights[1], len(y_val))
validation_weights[y_val==0] = val_weights[0]

In [11]:
# Encode the list of words to list of embedded arrays
def enc_question_pairs(qid_pairs, qid_question_dict, enc_map, swap=False):

    # encode single question pair
    def enc_question_pair(pair):

        # This array is a local cache that identify those non-encoded rare words. 
        # We'll assign identical index (in this question pair) for each of rare words
        rare_words = []

        def enc_question(qid):

            def map_word(word):
                if word in enc_map:
                    return enc_map[word]
                elif word in rare_words:
                    idx = np.where(np.array(rare_words)==word)[0][0]
                    return enc_map['<RARE' + str(idx) + '>']
                else:
                    rare_words.append(word)
                    return enc_map['<RARE' + str(len(rare_words)-1) + '>']

            question = qid_question_dict[qid]
            return [map_word(word) for word in question]
        
        if swap:
            return [enc_question(pair[1]) , enc_question(pair[0])]
        else:
            return [enc_question(pair[0]) , enc_question(pair[1])]
    

    return np.array([enc_question_pair(pair) for pair in qid_pairs])

In [12]:
'''
Generate training/validation data

Pseudo code:

1. Random assign a vector for <RARE> tags, this vector values only used in this batch
2. Generate positive samples 
        if same_question_ratio is set:
            Random assign partial positive samples as same question pairs (i.e. [qid_A, qid_A] )
        the remaining ratio of samples would be picked from pos_pairs
3. Generate negative samples 
        if random_negative_samples_ratio is set:
            Random assigning qid pairs as negative sample. (The number of negative samples is decided by "pos_ratio" parameter)
        the remaining ratio of samples would be picked from neg_pairs
3. Encode all question pairs.
        The same RARE word in a single question pair is gauranteed to be mapped to same vector.   
4. Shuffle the order of all question pairs.
5. Embed the question pairs.
'''
def gen_batch_data(pos_pairs, neg_pairs, batch_size, pos_ratio, swap=False, N_gram=None):
    
    global embedding_matrix
    
    all_id_list = list(qid_question_dict.keys())
    
    def gen_rnd_idx(list_data):
        return random.randint(0,len(list_data)-1)
    
    def random_pick_from(list_data):
        return list_data[gen_rnd_idx(list_data)]
    
    def random_assign_rare_vectors():
        rare_idxes = [enc_map['<RARE'+str(i)+'>'] for i in range(30)]
        for i in rare_idxes:
            embedding_matrix[i] = np.random.rand(300)
            
    def gen_shuffle_idxes(num):
        a = np.arange(num)
        random.shuffle(a)
        return a
    
    def gen_positive_qid_pairs(pos_pairs, N):
        
        N_pos_pairs = [pos_pairs[random.randint(0,pos_pairs.shape[0]-1)] for i in range(N)]
        
        if same_question_ratio!=0:
            same_question_count = round(same_question_ratio*N)
            replaced_by_same_question_idxes = [gen_rnd_idx(N_pos_pairs) for i in range(same_question_count)]
            for i in replaced_by_same_question_idxes:
                rnd_qid = random_pick_from(all_id_list)
                N_pos_pairs[i] = [rnd_qid,rnd_qid]
        
        return N_pos_pairs
        
    def gen_negative_qid_pairs(neg_pairs, N):
        
        '''
        TODO:
            Maybe we still need to check if generated question pair belongs to duplicated question samples.
            Although I think this is not very necessay, but we still can give it a try if we have time.
        '''
        
        N_neg_pairs = [random_pick_from(neg_pairs) for i in range(N)]
        
        if random_negative_samples_ratio!=0:
            random_sample_count = round(random_negative_samples_ratio*N)
            replaced_by_random_idxes = [gen_rnd_idx(N_neg_pairs) for i in range(random_sample_count)]
            for i in replaced_by_random_idxes:
                N_neg_pairs[i] = [random_pick_from(all_id_list),random_pick_from(all_id_list)]
        
        return N_neg_pairs
    
    # gaurantee <RARE> would be different in each batch
    random_assign_rare_vectors()
    
    # generate negative samples to match the given pos:neg ratio
    
    pos_count = int(batch_size*training_pos_ratio)
    neg_count = int( ((1-pos_ratio)/pos_ratio) * pos_count )
    
    pos_pairs = gen_positive_qid_pairs(pos_pairs, pos_count)
    neg_pairs = gen_negative_qid_pairs(neg_pairs, neg_count)
    
    pos = enc_question_pairs(pos_pairs, qid_question_dict, enc_map, swap=swap)
    neg = enc_question_pairs(neg_pairs, qid_question_dict, enc_map, swap=swap)
    X = np.vstack([pos,neg])
    
    # create y
    
    y = np.zeros(X.shape[0])
    y[:pos.shape[0]] = 1
    
    # shuffle
    
    shuffle_idxes = gen_shuffle_idxes(y.shape[0])
    X = X[shuffle_idxes]
    y = y[shuffle_idxes]
    
    # embed X
    if N_gram==None:
        embed_q1 = embedding_matrix[X[:,0,:]]
        embed_q2 = embedding_matrix[X[:,1,:]]
    else:
        embed_q1 = N_gram_embedding(X[:,0,:],N_gram)
        embed_q2 = N_gram_embedding(X[:,1,:],N_gram)
        
    weights = np.zeros(y.shape[0])
    weights[y==0] = training_weights[0]
    weights[y==1] = training_weights[1]
    
    return [embed_q1,embed_q2], y, weights

In [13]:
# Calculate the class weights to simulate the class disribution of testing samples
#
# These numbers and formula is got from here: 
#     https://www.kaggle.com/lystdo/quora-question-pairs/lstm-with-word2vec-embeddings#175198

test_set_pos_label_ratio = 0.1746
training_pos_ratio = 0.5

same_question_ratio = 0.1 # a training hyper paramter to add testcases like [A,A] 
random_negative_samples_ratio = 0.01

training_weights = {
    0: (1-test_set_pos_label_ratio) / (1-training_pos_ratio),
    1: test_set_pos_label_ratio/training_pos_ratio
}

In [14]:
import random

batch_size = 128

def batch_generator(pos_pairs, neg_pairs, switch_position=False):
    
    while True:
        
        if switch_position and random.randint(0,1)==0:
            switch_position = True
        
        yield gen_batch_data(pos_pairs, neg_pairs, batch_size, training_pos_ratio, swap=switch_position, N_gram=N_gram)
        

# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)
# ]
    
# try:
    
#     model.fit_generator(batch_generator(positive_question_pairs, negative_question_pairs, switch_position=True),
#                         steps_per_epoch=50,
#                         class_weight=weights,
#                         epochs=1000,
#                         validation_data=(X_val,y_val,validation_weights), 
#                         callbacks=callbacks)
    
# except KeyboardInterrupt:
#     print('\nEarly stopped by user')

#### Create model

In [48]:
import tensorflow as tf
from tensorflow.contrib.rnn import GRUCell, DropoutWrapper, MultiRNNCell

# clear all preset variables and scopes

tf.reset_default_graph()

# hyper-parameters that should be passed as function argument

vocab_size = len(enc_map)
vocab_dim = 300 # embedded by GloVe with dimension=300
text_length = 30 # how many words are we going to feed in one time
time_stamps = text_length + N_gram - 1

# hyper-params

RNN_cell = GRUCell # or, LSTMCell

meaning_feature_size = 128
fc1_size = 32

semantic_compressor_num_layers = 1
semantic_compressor_dropout_rate = 0.05

meaning_analyzer_num_layers = 1
meaning_analyzer_dropout_rate = 0.05

fc1_dropout_rate = 0.05
fc2_dropout_rate = 0.05

noise_std = 0.001

# start build model

sess = tf.InteractiveSession()

q1 = tf.placeholder(tf.float32, shape=[None, time_stamps, N_gram, vocab_dim], name='q1')
q2 = tf.placeholder(tf.float32, shape=[None, time_stamps, N_gram, vocab_dim], name='q2')
ground_truth_labels = tf.placeholder(tf.float32, shape=[None, 1], name='GT')
class_weights = tf.placeholder(tf.float32, shape=[None, 1], name='sample_weights')

# add noise

noise = tf.random_normal(shape=tf.shape(q1), mean=0.0, stddev=noise_std) 
noised1 = noise + q1
noised2 = noise + q2

reshaped1 = tf.reshape(noised1, shape=[-1, time_stamps, N_gram*vocab_dim], name='reshape_input_1')
reshaped2 = tf.reshape(noised2, shape=[-1, time_stamps, N_gram*vocab_dim], name='reshape_input_2')

# compress the N-gram vector back to 1-word vector (i.e. 300 dimensions)

def semantic_compressor(inputs):
    with tf.variable_scope("semantic_compressor"):
        dropout_rate = tf.Variable(semantic_compressor_dropout_rate, name='semantic_rnn_dropout')
        semantic_compressor_cell = RNN_cell(vocab_dim)
        semantic_compressor_cell = DropoutWrapper(semantic_compressor_cell, output_keep_prob=dropout_rate)
        semantic_compressor_cells = MultiRNNCell([semantic_compressor_cell] * semantic_compressor_num_layers)
        output, state = tf.nn.dynamic_rnn(semantic_compressor_cells, inputs, dtype=tf.float32)
#         tf.summary.histogram('semantic_compressor_cells', semantic_compressor_cells) # TODO: find a way to visualize RNN weights
        tf.summary.histogram('semantic_compressor_output', output)
        return output

with tf.variable_scope("compress_inputs") as scope:
    compressed_input1 = semantic_compressor(reshaped1)
    scope.reuse_variables()
    compressed_input2 = semantic_compressor(reshaped2)
    
    tf.summary.histogram('compressed_input1', compressed_input1)
    tf.summary.histogram('compressed_input2', compressed_input2)

# concat the two questions and use RNN extract its meaning features

with tf.variable_scope("analyze_meaning_features"):
    concat_inputs = tf.concat([compressed_input1, compressed_input2], axis=2, name='concat_compressed_inputs')

    with tf.variable_scope("meaning_analyzer"):
        meaning_analyzer_cell = RNN_cell(meaning_feature_size)
        meaning_analyzer_cell = DropoutWrapper(meaning_analyzer_cell, output_keep_prob=meaning_analyzer_dropout_rate)
        meaning_analyzer_cells = MultiRNNCell([meaning_analyzer_cell] * meaning_analyzer_num_layers)
        compressed_meaning, state = tf.nn.dynamic_rnn(meaning_analyzer_cells, concat_inputs, dtype=tf.float32)
        
    # we don't care about the sequences distributed over time returned by RNN, just want the last time stamp

    def get_last_time_stamp(inputs):
        inputs = tf.transpose(inputs, [1, 0, 2])
        last = tf.gather(inputs, int(inputs.get_shape()[0]) - 1)
        return last
        
    compressed_meaning = get_last_time_stamp(compressed_meaning)
    
#     tf.summary.histogram('meaning_analyzer_cells', meaning_analyzer_cells) # TODO: find a way to visualize RNN weights
    tf.summary.histogram('compressed_meaning', compressed_meaning)

# fc1
with tf.variable_scope("fc1"):

    fc_serial = 1
    prev_tensor = compressed_meaning

    fc1 = tf.contrib.layers.batch_norm(prev_tensor)
#     fc1 = tf.nn.relu(fc1, name='pre_act_fc'+str(fc_serial))
    fc1 = tf.layers.dropout(fc1, fc1_dropout_rate)
    fc1 = tf.contrib.layers.fully_connected(fc1, fc1_size)
    
    tf.summary.histogram('fc1_result', fc1)

# fc2
with tf.variable_scope("fc2"):

    fc_serial = 2
    prev_tensor = fc1

    fc2 = tf.contrib.layers.batch_norm(prev_tensor)
#     fc2 = tf.nn.relu(fc2, name='pre_act_fc'+str(fc_serial))
    fc2 = tf.layers.dropout(fc2, fc2_dropout_rate)
    fc2 = tf.contrib.layers.fully_connected(fc2, 1)

out = tf.sigmoid(fc2, name='sigmoid')
tf.summary.histogram('final_sigmoid', out)


<tf.Tensor 'final_sigmoid:0' shape=() dtype=string>

In [49]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean( #reduce_logsumexp
        tf.nn.sigmoid_cross_entropy_with_logits(logits=out, labels=ground_truth_labels, name='sigmoid_cross_entropy'))
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

with tf.name_scope('log_loss'):
    log_loss = tf.reduce_mean(tf.losses.log_loss(ground_truth_labels, out, weights=class_weights))
tf.summary.scalar('log_loss', log_loss)

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<tf.Tensor 'log_loss_1:0' shape=() dtype=string>

In [50]:
merge_summary = tf.summary.merge_all()

writer = tf.summary.FileWriter('./tmp/test/0')
writer.add_graph(sess.graph)

In [51]:
sess.run(tf.global_variables_initializer())

steps_pre_epoch = 1

training_generator = batch_generator(positive_question_pairs, negative_question_pairs, switch_position=True)

for i in range(1000):

    # training

    batch = next(training_generator)
    feed_dict={
        q1: batch[0][0], 
        q2: batch[0][1], 
        ground_truth_labels: batch[1].reshape(-1,1), 
        class_weights: batch[2].reshape(-1,1)
    }
    _, loss_val = sess.run([train_step, log_loss], feed_dict=feed_dict)

    if i%30==0:
        print('Training loss at ', i, ': ', loss_val)
    
    # write to tensor board
    
    if i%5==0 and i!=0:
        summary = sess.run(merge_summary, feed_dict=feed_dict)
        writer.add_summary(summary,i)
    
    if i%30==0 and i!=0:
        pass
#         # validation 
#         val_dict={
#             q1: X_val[0][100],
#             q2: X_val[1][100], 
#             ground_truth_labels: y_val.reshape(-1,1)[100], 
#             class_weights: validation_weights.reshape(-1,1)[100]
#         }
#         val_log_loss = sess.run([log_loss], feed_dict=val_dict)


#         print('validation log_loss=', val_log_loss)



Training loss at  0 :  0.93714
Training loss at  30 :  0.867157
Training loss at  60 :  0.742115
Training loss at  90 :  0.718826
Training loss at  120 :  0.711182
Training loss at  150 :  0.698874
Training loss at  180 :  0.697731
Training loss at  210 :  0.697504
Training loss at  240 :  0.69692
Training loss at  270 :  0.695463
Training loss at  300 :  0.693158


KeyboardInterrupt: 

Calculate the ROC_AUC score

In [ ]:
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

pred = model.predict(X_val)

# pred_bin = np.zeros(len(pred))
# pred_bin[(pred>0.5).reshape(len(pred))] = 0.999999
# pred_bin[(pred<=0.5).reshape(len(pred))] = 0.000001

pred[pred>0.999] = 0.999999
pred[pred<=0.001] = 0.000001

print('log_loss=',log_loss(y_val, pred, sample_weight=validation_weights))

print('roc_auc=',roc_auc_score(y_val, pred, sample_weight=validation_weights))

In [ ]:
# model_name = 'HubertLin_naive_LSTM_dynamic_training_tri_earlystop'
# model.save('../model/'+model_name+'.model')

## customized testcase

In [50]:
# from keras.models import load_model
# model = load_model('../model/HubertLin_naive_LSTM_alot_dropouts.model')

In [18]:
def enc_question_pair(question_pair_list, enc_map, capsulate_st_ed=False):

    def enc_question(question):

        if type(question)!=str or question=="":
            if capsulate_st_ed:
                return [enc_map['<ST>'], enc_map['<ED>']]
            else:
                return []

        def map_wrod(word):
            if word in enc_map:
                return enc_map[word]
            elif word in rare_words:
                idx = np.where(np.array(rare_words)==word)[0][0]
                return enc_map['<RARE' + str(idx) + '>']
            else:
                rare_words.append(word)
                return enc_map['<RARE' + str(len(rare_words)-1) + '>']

        # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
        separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|$)'
        single_word = '[^\s\-]+' # non-empty is enough here

        words_list = re.findall(single_word+separator, question)

        if capsulate_st_ed:
            return [enc_map['<ST>']] + [map_wrod(word) for word in words_list] + [enc_map['<ED>']] 
        else:
            return [map_wrod(word) for word in words_list]
        
    # This array is a local cache that identify those non-encoded rare words. 
    # We'll assign identical index (in this question pair) for each of rare words
    rare_words = []
    
    return enc_question(question_pair_list[0]) , enc_question(question_pair_list[1])
    
    
def dec_question(question, dec_map):
    return [dec_map[enc_value] for enc_value in question]

def clip_length(data, clip_st_ed=False):
    
    clip_len = 30
    
    if clip_st_ed:
        data[0] = eval(data[0])[1:-1]
        data[1] = eval(data[1])[1:-1]
    
    if len(data[0])<clip_len:
        data[0] = data[0] + [enc_map['<ED>']] * (30-len(data[0]))
    else:
        data[0] = data[0][:30]
        
    if len(data[1])<clip_len:
        data[1] = data[1] + [enc_map['<ED>']] * (30-len(data[1]))
    else:
        data[1] = data[1][:30]
        
    return data

In [19]:
import numpy as np
import pandas as pd
import re
import pickle

dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))

testcases = [
    {
        'q1': 'I am a good man',
        'q2': 'A good man is me'
    },
    {
        'q1': 'The <RARE0> is totally fucking awful',
        'q2': 'The <RARE0> is the best thing I have ever seen'
    },
    {
        'q1': 'The <RARE0> is the best thing I have ever seen',
        'q2': 'The <RARE0> is totally fucking awful',
    },
    {
        'q1': 'How <RARE0> can be used in cooking',
        'q2': 'Can we find <RARE1> on the moon'
    },
    {
        'q1': 'A bird is flying in the sky',
        'q2': 'My bird is flying over the ocean'
    },
    {
        'q1': 'What is your favorite food',
        'q2': 'Which is your favorite one'
    },
    {
        'q1': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
        'q2': '<RARE6> <RARE7> <RARE8> <RARE9> <RARE10> <RARE11> ',
    },
    {
        'q1': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
        'q2': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
    },
    {
        'q1': ' what a <RARE0> program I have never seen such beatiful code ever',
        'q2': ' what a <RARE0> program I have never seen such beatiful code ever',
    },
    {
        'q1': ' what a <RARE0> program I have never seen such beatiful code ever',
        'q2': ' what a <RARE1> program I have never seen such beatiful code ever',
    },
]

for testcase in testcases:
    
    enc1, enc2 = enc_question_pair([testcase['q1'],testcase['q2']], enc_map)
    enc1, enc2 = clip_length([enc1,enc2])
    print(' '.join(dec_question(enc1,dec_map)))
    print(' '.join(dec_question(enc2,dec_map)))

    if N_gram==None:
        emb1 = embedding_matrix[np.array([enc1])]
        emb2 = embedding_matrix[np.array([enc2])]
    else:
        emb1 = N_gram_embedding(np.array([enc1]), N_gram)
        emb2 = N_gram_embedding(np.array([enc2]), N_gram)

    pred = model.predict([emb1,emb2])
    print('Predicting is same question proba =', pred)
    print('')


I am a good man <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
A good man is me <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Predicting is same question proba = [[ 0.20090792]]

The <RARE0> is totally fucking awful <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
The <RARE0> is the best thing I have ever seen <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Predicting is same question proba = [[ 0.2374358]]

The <RARE0> is the best thing I have ever seen <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
The <RARE0> is totally fucking awful <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Pred

## Try to predict some testing data

In [40]:
df_test = pickle.load(open('../dataset/processed/df_test_hubertLin_version.pkl', 'rb'))

In [46]:
a = df_test.ix[186461]
q1 = dec_question(a['question1'], dec_map)
q2 = dec_question(a['question2'], dec_map)
print(q1)
print(q2)

emb1 = embedding_matrix[np.array([a['question1']])]
emb2 = embedding_matrix[np.array([a['question2']])]
pred = model.predict([emb1, emb2])
print('\nPredicting is same question proba =', pred)

['What', 'is', 'customer', 'lifetime', '<RARE0>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>']
['What', 'is', 'customer', 'lifetime', 'value', 'in', '<RARE1>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>']

Predicting is same question proba = [[ 0.60896122]]


## Prediction time

In [ ]:
from keras.models import load_model

try:
    model==None
except:
    model = load_model('../model/'+model_name+'.model')

df_test = pickle.load(open('../dataset/processed/df_test_hubertLin_version.pkl', 'rb'))

In [47]:
%%time

partition_size = 10000

def predict(i,q):
    
    if i%10 == 0:
        print(i*partition_size, '/', len(df_test))
    
    x1 = np.array(list(q['question1']))
    x2 = np.array(list(q['question2']))
    
    x1 = embedding_matrix[x1]
    x2 = embedding_matrix[x2]
    
    return model.predict([x1,x2])

partition_len = len(df_test)//partition_size +1
result = [predict(i,df_test.iloc[i*partition_size:(i+1)*partition_size]) for i in range(partition_len)]
con = np.concatenate(result)
df_result = pd.DataFrame({'test_id':np.arange(len(con)),'is_duplicate':con.reshape(len(con))}, columns=['test_id','is_duplicate'])
df_result.to_csv('../result/prediction.csv', index=False)

0 / 2345796
100000 / 2345796
200000 / 2345796
300000 / 2345796
400000 / 2345796
500000 / 2345796
600000 / 2345796
700000 / 2345796
800000 / 2345796
900000 / 2345796
1000000 / 2345796
1100000 / 2345796
1200000 / 2345796
1300000 / 2345796
1400000 / 2345796
1500000 / 2345796
1600000 / 2345796
1700000 / 2345796
1800000 / 2345796
1900000 / 2345796
2000000 / 2345796
2100000 / 2345796
2200000 / 2345796
2300000 / 2345796
Wall time: 48min 13s


In [51]:
if len(df_result)!=2345796:
    print('Your result prediction count is not fit to the testing data length 2345796 , yours:', len(df_result))
else:
    print('Prediction success')

Prediction success


## visualize the prediction result

In [54]:
over = df_test[df_result['is_duplicate']>0.8]

c = 0
for i,s in over.iterrows():
    print(' '.join(dec_question(s['question1'],dec_map)))
    print(' '.join(dec_question(s['question2'],dec_map)))
    print(df_result.ix[i]['is_duplicate'])
    print('')
    c+=1
    if c>5:
        break

Can a vacuum cleaner concentrate suck your eye out if it is pressed against your <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Could a vacuum cleaner suck get your eye out if directly pressed on the <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.813005

Is web development just building <RARE0> best you get a web developer job if you know how to make a <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Is web development just building <RARE0> Can you get a backend web developer job if you know how year make a <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.966372

How do I overcome my shyness with <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
How do you overcome being <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.865856

If I like a comment to a computer post b